In [25]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/alzatem204/Tennis-Predictions/main/General_Data_ATP_UK.csv"
response = requests.get(url)

with open("General_Data_ATP_UK.csv", "wb") as f:
    f.write(response.content)

df = pd.read_csv("General_Data_ATP_UK.csv")
display(df.head())

/tmp/ipython-input-25-602830675.py:10: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("General_Data_ATP_UK.csv")


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Total Games,Partido,Sets,Score,Game Difference,Games Winner,Games Loser,Total Sets,Sets Difference,Year
0,34.0,Eastbourne,Eastbourne International,6/28/2025,ATP250,Outdoor,Grass,The Final,3,Fritz T.,...,19,Fritz T. vs. Brooksby J.,2 - 0,7-5 6-1,7,13,6,2,2,2025
1,35.0,Mallorca,Mallorca Championships,6/28/2025,ATP250,Outdoor,Grass,The Final,3,Griekspoor T.,...,25,Griekspoor T. vs. Moutet C.,2 - 0,7-5 7-6,3,14,11,2,2,2025
2,34.0,Eastbourne,Eastbourne International,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Brooksby J.,...,33,Brooksby J. vs. Humbert U.,2 - 1,6-7 6-4 6-4,3,18,15,3,1,2025
3,34.0,Eastbourne,Eastbourne International,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Fritz T.,...,25,Fritz T. vs. Davidovich Fokina A.,2 - 1,6-3 3-6 6-1,5,15,10,3,1,2025
4,35.0,Mallorca,Mallorca Championships,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Griekspoor T.,...,20,Griekspoor T. vs. Auger-Aliassime F.,2 - 0,6-4 6-4,4,12,8,2,2,2025


In [26]:
from collections import defaultdict

# Eliminar columnas irrelevantes (mantener 'Date')
columns_to_remove = [
    'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5',
    'Wsets', 'Lsets', 'Score', 'Sets', 'Total Sets',
    'Game Difference', 'Games Winner', 'Games Loser', 'Total Games', 'Sets Difference',
    'Comment', 'Partido',
    'B365W', 'B365L', 'PSW', 'PSL', 'MaxW', 'MaxL',
    'Best Ranking?', 'Best Odds?', 'ATP', 'Year'
]
df = df.drop(columns=columns_to_remove, errors='ignore')

# Convertir y ordenar por fecha
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Winner', 'Loser']).sort_values('Date')

# Inicializar listas
wh2h_list = []
lh2h_list = []
wh2h_surf_list = []
lh2h_surf_list = []

# Diccionarios de historial
h2h_record = defaultdict(int)
h2h_surface_record = defaultdict(int)

# Calcular métricas por fila
for _, row in df.iterrows():
    w = row['Winner']
    l = row['Loser']
    surface = row['Surface']

    # H2H general
    wh2h = h2h_record[(w, l)]
    lh2h = h2h_record[(l, w)]

    # H2H por superficie
    wh2h_surf = h2h_surface_record[(w, l, surface)]
    lh2h_surf = h2h_surface_record[(l, w, surface)]

    # Guardar métricas
    wh2h_list.append(wh2h)
    lh2h_list.append(lh2h)
    wh2h_surf_list.append(wh2h_surf)
    lh2h_surf_list.append(lh2h_surf)

    # Actualizar historiales
    h2h_record[(w, l)] += 1
    h2h_surface_record[(w, l, surface)] += 1

# Agregar al dataframe
df['Wh2h'] = wh2h_list
df['Lh2h'] = lh2h_list
df['Wh2h_surf'] = wh2h_surf_list
df['Lh2h_surf'] = lh2h_surf_list


# Verifica dimensiones y columnas finales
print("Filas totales:", df.shape[0])
print("Columnas:", df.columns.tolist())

Filas totales: 26423
Columnas: ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'Wh2h', 'Lh2h', 'Wh2h_surf', 'Lh2h_surf']


In [27]:
# Convertir y ordenar por fecha
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Winner', 'Loser']).sort_values('Date')

# Inicializar estructuras para almacenar métricas
whist_list = []
lhist_list = []
whist_surf_list = []
lhist_surf_list = []

# Diccionarios para llevar cuenta del historial
player_wins = defaultdict(int)
player_losses = defaultdict(int)
player_surface_wins = defaultdict(int)
player_surface_losses = defaultdict(int)

# Calcular métricas por fila
for _, row in df.iterrows():
    w = row['Winner']
    l = row['Loser']
    surface = row['Surface']

    # Historial general
    whist = player_wins[w] - player_losses[w]
    lhist = player_wins[l] - player_losses[l]

    # Historial por superficie
    whist_surf = player_surface_wins[(w, surface)] - player_surface_losses[(w, surface)]
    lhist_surf = player_surface_wins[(l, surface)] - player_surface_losses[(l, surface)]

    # Guardar en listas
    whist_list.append(whist)
    lhist_list.append(lhist)
    whist_surf_list.append(whist_surf)
    lhist_surf_list.append(lhist_surf)

    # Actualizar historiales tras el partido
    player_wins[w] += 1
    player_losses[l] += 1
    player_surface_wins[(w, surface)] += 1
    player_surface_losses[(l, surface)] += 1

# Agregar columnas al dataframe
df['Whist'] = whist_list
df['Lhist'] = lhist_list
df['Whist_surf'] = whist_surf_list
df['Lhist_surf'] = lhist_surf_list

# Verifica dimensiones y columnas finales
print("Filas totales:", df.shape[0])
print("Columnas:", df.columns.tolist())

Filas totales: 26423
Columnas: ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'Wh2h', 'Lh2h', 'Wh2h_surf', 'Lh2h_surf', 'Whist', 'Lhist', 'Whist_surf', 'Lhist_surf']


In [28]:
from collections import  deque


# Asegúrate de tener la columna 'Date' en formato datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Eliminar filas sin fecha o jugadores
df = df.dropna(subset=['Date', 'Winner', 'Loser'])

# Ordenar por fecha para procesar cronológicamente
df = df.sort_values('Date').reset_index(drop=True)

# Inicializar listas de métricas
w10_list = []
l10_list = []
w10_surf_list = []
l10_surf_list = []

# Diccionarios para mantener historial de los últimos 10 partidos (total y por superficie)
last_matches = defaultdict(lambda: deque(maxlen=10))
last_surface_matches = defaultdict(lambda: deque(maxlen=10))

# Calcular métricas fila por fila
for _, row in df.iterrows():
    w = row['Winner']
    l = row['Loser']
    surface = row['Surface']

    # Últimos 10 resultados generales
    w_last = list(last_matches[w])
    l_last = list(last_matches[l])
    w10 = w_last.count('W') - w_last.count('L')
    l10 = l_last.count('W') - l_last.count('L')

    # Últimos 10 resultados en esa superficie
    w_surf_last = list(last_surface_matches[(w, surface)])
    l_surf_last = list(last_surface_matches[(l, surface)])
    w10_surf = w_surf_last.count('W') - w_surf_last.count('L')
    l10_surf = l_surf_last.count('W') - l_surf_last.count('L')

    # Agregar a las listas
    w10_list.append(w10)
    l10_list.append(l10)
    w10_surf_list.append(w10_surf)
    l10_surf_list.append(l10_surf)

    # Actualizar historial con el resultado actual
    last_matches[w].append('W')
    last_matches[l].append('L')
    last_surface_matches[(w, surface)].append('W')
    last_surface_matches[(l, surface)].append('L')

# Agregar columnas al DataFrame
df['W10'] = w10_list
df['L10'] = l10_list
df['W10_surf'] = w10_surf_list
df['L10_surf'] = l10_surf_list

# Verifica dimensiones y columnas finales
print("Filas totales:", df.shape[0])
print("Columnas:", df.columns.tolist())

Filas totales: 26423
Columnas: ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'Wh2h', 'Lh2h', 'Wh2h_surf', 'Lh2h_surf', 'Whist', 'Lhist', 'Whist_surf', 'Lhist_surf', 'W10', 'L10', 'W10_surf', 'L10_surf']


In [29]:
df_swap_ready = df.copy()

# Agregar columna target (1 si el jugador en columna 'Winner' ganó)
df_swap_ready['target'] = 1

# Crear duplicado para intercambio simétrico
df_flipped = df_swap_ready.copy()

# Definir columnas a intercambiar (ganador ↔ perdedor)
swap_columns = {
    'Winner': 'Loser',
    'Loser': 'Winner',
    'WRank': 'LRank',
    'LRank': 'WRank',
    'WPts': 'LPts',
    'LPts': 'WPts',
    'AvgW': 'AvgL',
    'AvgL': 'AvgW',
    'Wh2h': 'Lh2h',
    'Lh2h': 'Wh2h',
    'Wh2h_surf': 'Lh2h_surf',
    'Lh2h_surf': 'Wh2h_surf',
    'Whist': 'Lhist',
    'Lhist': 'Whist',
    'Whist_surf': 'Lhist_surf',
    'Lhist_surf': 'Whist_surf',
    'W10': 'L10',
    'L10': 'W10',
    'W10_surf': 'L10_surf',
    'L10_surf': 'W10_surf'
}

# Aplicar intercambio
df_flipped = df_flipped.rename(columns=swap_columns)

# Actualizar la columna target para el duplicado
df_flipped['target'] = 0

# Concatenar original + invertido
df_balanced_full = pd.concat([df_swap_ready, df_flipped], ignore_index=True)

# Mezclar filas aleatoriamente (importante para entrenamiento)
df_balanced_full = df_balanced_full.sample(frac=1, random_state=42).reset_index(drop=True)

# Verifica dimensiones y columnas finales
print("Filas totales:", df_balanced_full.shape[0])
print("Columnas:", df_balanced_full.columns.tolist())

# Guardar
df_balanced_full.to_csv("ATP_model_ready.csv", index=False)

Filas totales: 52846
Columnas: ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'Wh2h', 'Lh2h', 'Wh2h_surf', 'Lh2h_surf', 'Whist', 'Lhist', 'Whist_surf', 'Lhist_surf', 'W10', 'L10', 'W10_surf', 'L10_surf', 'target']
Filas totales: 52846
Columnas: ['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'Wh2h', 'Lh2h', 'Wh2h_surf', 'Lh2h_surf', 'Whist', 'Lhist', 'Whist_surf', 'Lhist_surf', 'W10', 'L10', 'W10_surf', 'L10_surf', 'target']


In [30]:
df_balanced_full.head()

,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,...,Lh2h_surf,Whist,Lhist,Whist_surf,Lhist_surf,W10,L10,W10_surf,L10_surf,target
0,Auckland,ASB Classic,2016-01-11,ATP250,Outdoor,Hard,1st Round,3.0,Ramos-Vinolas A.,De Bakker T.,...,0,-8,-3,-2,-2,-4,-3,-4,-2,0
1,Memphis,Memphis Open,2015-02-13,ATP250,Indoor,Hard,2nd Round,3.0,Groth S.,Anderson K.,...,0,2,3,2,3,2,2,2,2,0
2,Los Cabos,Abierto Mexicano Mifel,2018-07-31,ATP250,Outdoor,Hard,1st Round,3.0,Fritz T.,Kokkinakis T.,...,1,-5,-9,1,-3,-2,-4,2,-2,1
3,St. Petersburg,St. Petersburg Open,2017-09-24,ATP250,Indoor,Hard,The Final,3.0,Fognini F.,Dzumhur D.,...,0,18,2,3,6,4,6,4,6,0
4,Tokyo,Rakuten Japan Open Tennis Championships,2017-10-07,ATP500,Outdoor,Hard,Semifinals,3.0,Schwartzman D.,Goffin D.,...,1,0,55,5,33,4,6,4,6,0


In [31]:
from sklearn.preprocessing import LabelEncoder

columns_to_remove = ['Date']

df_balanced_full = df_balanced_full.drop(columns=columns_to_remove, errors='ignore')

categorical_cols = ['Location', 'Tournament', 'Series', 'Court', 'Surface', 'Round',
                    'Best of', 'Winner', 'Loser']

# Rellenar valores nulos con 'Unknown'
for col in categorical_cols:
    df_balanced_full[col] = df_balanced_full[col].fillna('Unknown')

# Aplicar LabelEncoder a cada columna categórica
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_balanced_full[col] = le.fit_transform(df_balanced_full[col].astype(str))
    label_encoders[col] = le  # guardamos el encoder por si se necesita decodificar luego

In [32]:
df_balanced_full.head()

,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,...,Lh2h_surf,Whist,Lhist,Whist_surf,Lhist_surf,W10,L10,W10_surf,L10_surf,target
0,7,4,0,1,2,0,2,577,147,55.0,...,0,-8,-3,-2,-2,-4,-3,-4,-2,0
1,54,77,0,0,2,1,2,272,13,70.0,...,0,2,3,2,3,2,2,2,2,0
2,46,6,0,1,2,0,2,227,356,65.0,...,1,-5,-9,1,-3,-2,-4,2,-2,1
3,85,115,0,0,2,7,2,221,190,29.0,...,0,18,2,3,6,4,6,4,6,0
4,90,100,1,1,2,6,2,627,255,29.0,...,1,0,55,5,33,4,6,4,6,0


In [33]:
df_balanced_full.to_csv("ATP_UK_Metrics_Encoded.csv", index=False)